In [1]:
import pandas as pd

src_df = pd.read_excel('r7_r8_marked_result.xlsx')

In [2]:
len(src_df)

1261

In [12]:
# prompt 变更的文本，分析差异的地方

from openai import OpenAI
import os

client = OpenAI(max_retries=5, api_key=os.getenv('OPENAI_API_KEY'))

def get_diff(r7_text, r8_text):
    completion = client.chat.completions.create(
      model="gpt-4o",
      temperature=0.2,
      messages=[
        {"role": "system", "content": "你是中国民航局的法规制定人员，你有非常专业的民航知识。你现在需要帮我处理一些民航法规文件的事情。"},
        {"role": "user", "content": f"请找出R7和R8两个版本相关民航法规文本的区别, 请侧重寻找语义的差异。\
         请忽略以下差异： \
         1.中英文符号差异；\
         2.'民航局'、'中国民航局'、'局方'无差异，'地区管理局'和'民航地区管理局'无差异；\
         3.数值格式的差异(是否包含千位分隔符)，比如'7500元'和'7,500元'无差异， '1100小时'和'1,100小时'无差异；\
         4.单位叫法差异，比如°和度无差异；\
         5.专用名词后面是否带缩写，'专业术语'和'专业术语(缩写字母)无差异', 例如 名词1 和 名词1(缩写1)无差异；\
         6.标点符号的差异，比如；和。无差异；\
         7.书写者的习惯，比如'设备'和'设施'无差异、'运营'和'运行'无差异; \
         8.'CCAR数字'和'CCAR-数字'无差异；\
         9.中文连接词表达的差异，'或'和'或者'无差异； \
         \
         如果文本中提到以下内容，在两段文本中出现差异，请列举出来：\
         1.条款引用差异，条款示例：P章、'第121.xx条'(这里xx代表数字)、(a)、(1)、(i)；\
         2.数值差异，只统计数值不相等的差异，(例如‘2200小时'和'2,200小时'无差异, 但是‘1100'和'1,200'有差异) 数字示例：15.2米、50英尺、15°、1001小时、'1,001小时'；\
         3.日期差异；\
         4.规章文件引用的差异，规章文件示例：《民用航空器驾驶员合格审定规则》； \
         5.主体对象差异，主体对象示例：机组成员，乘务员，合格证持有人，局方（指民航局），机长，飞机上的各种系统，各种飞机型号；\
         \
         请严格按照以下格式输出： \
         差异1：差异点（只考虑上面提到的内容） \
         R7： 提取差异的内容，不需要是一段完整的话。如果没有，直接输入无 \
         R8： 提取差异的内容，不需要是一段完整的话。如果没有，直接输入无 \
         差异2：差异点（只考虑上面提到的内容） \
         R7： 提取差异的内容，不需要是一段完整的话。如果没有，直接输入无 \
         R8： 提取差异的内容，不需要是一段完整的话。如果没有，直接输入无 \
         如果没有其他差异，请不要继续输出 \
         如果有其他差异，请输出：\
         其他差异：上面提到的内容之外的差异（尽量简短描述，请不要重复上面提到的差异点）\
         如果两段文字只存在开始提到的需要忽略的差异，请直接输出：无内容差异 \
         以下是文本内容：\
         R7版本：{r7_text} \
         R8版本：{r8_text}"
        }
      ]
    )
    diff_content = completion.choices[0].message.content
    print('------------------------------------------------------------')
    print(diff_content)
    return diff_content

In [13]:
r7_text = """对于最大起飞全重 136,000 千克(含)以下的飞机，应当担任机长飞行一年以上，相应机长飞行经历时间不少于 300 小时，且总驾驶员飞行经历时间不少于 2200 小时；如不具有上述机长经历，则其总驾驶员飞行经历时间不得少于2700 小时，其中在组类 II 飞机上不少于 1000 小时，且作为操作驾驶员不少于400 个包括起飞和着陆的航段，其中在本机型上作为操作驾驶员不少于 200 个包括起飞着陆的航段；"""
r8_text = """对于最大起飞重量 136,000 千克(含)以下的飞机，应当担任机长飞行一年以上，相应机长飞行经历时间不少于300 小时，且总驾驶员飞行经历时间不少于 2,200 小时；如不具有上述机长经历，则其总驾驶员飞行经历时间不得少于2,700 小时，其中在组类 II 飞机上不少于 1,000 小时，且作为操作驾驶员不少于 400 个包括起飞和着陆的航段，其中在本机型上作为操作驾驶员不少于 200 个包括起飞着陆的航段；"""
test_diff = get_diff(r7_text, r8_text)

APIConnectionError: Connection error.

In [123]:
r7_text = """对于至少有两套能够提供不同跑道直线进近的可用进近设施的机场，其进近设施能提供直线非精密进近程序、直线类精密进近程序或直线 I 类精密进近程序，应选择两个服务于不同适用跑道的进近设施，在相应直线进近程序的决断高（DH）或最低下降高（MDH）较高值上增加 60 米(200 英尺)，在能见度较高值上增加 800 米(1/2 英里)。"""
r8_text = """对于至少有两套能够提供不同跑道直线进近的可用进近设施的机场，其进近设施能提供直线非精密进近程序(NPA)、直线有垂直引导的进近程序(APV)或者直线 I 类精密进近程序(PA)，应当选择两个服务于不同适用跑道的进近设施，在相应直线进近程序的决断高(DH)或者最低下降高(MDH)较高值上增加 60 米(200 英尺)，在能见度(VIS)较高值上增加 800 米(1/2 英里)。"""
test_diff = get_diff(r7_text, r8_text)

差异1：规章文件引用的差异
R7：无
R8：《民用航空器驾驶员合格审定规则》

差异2：主体对象差异
R7：直线非精密进近程序、直线类精密进近程序
R8：直线非精密进近程序(NPA)、直线有垂直引导的进近程序(APV)

其他差异：上面提到的内容之外的差异
R8版本中增加了缩写形式，例如NPA、APV、PA、DH、MDH、VIS。


In [131]:
r7_text = """工程技术部门、维修计划和控制部门、质量部门中从事工程技术管理、维修质量管理和飞机放行的人员应当具有《民用航空器维修人员执照管理规则》（CCAR65）的《维修人员执照》，其专业和机型类别应当与所从事的工作相适应；"""
r8_text = """工程技术部门、维修计划和控制部门、质量安全部门的主管应当持有《民用航空器维修人员执照管理规则》(CCAR-65)的民用航空器维修人员执照，具备运输类飞机机型签署经历，并具备维修工程管理经验；"""
test_diff = get_diff(r7_text, r8_text)

------------------------------------------------------------
差异1：主体对象差异
R7：从事工程技术管理、维修质量管理和飞机放行的人员
R8：质量安全部门的主管

差异2：规章文件引用的差异
R7：《维修人员执照》
R8：民用航空器维修人员执照

差异3：数值差异
R7：无
R8：运输类飞机机型签署经历

差异4：条款引用差异
R7：无
R8：无

差异5：日期差异
R7：无
R8：无

其他差异：R8版本增加了对主管人员的具体要求，包括具备运输类飞机机型签署经历和维修工程管理经验。


In [4]:
src_df.head(5)

,text_level,r7_text_key,text_change_type,r8_text_key,similar_score,r7_text_content,r8_text_content,diff_content
0,section,121.175,删除,NaN,NaN,NaN,NaN,NaN
1,section,121.177,删除,NaN,NaN,NaN,NaN,NaN
2,section,121.179,删除,NaN,NaN,NaN,NaN,NaN
3,section,121.181,删除,NaN,NaN,NaN,NaN,NaN
4,section,121.183,删除,NaN,NaN,NaN,NaN,NaN


In [5]:
src_df['text_change_type'].unique()

array(['删除', '新增', '节标题更新', '节头部正文内容更新', '变更'], dtype=object)

In [6]:
len(src_df)

1261

In [ ]:
src_df['diff_content'] = src_df.apply(lambda row: get_diff(row['r7_text_content'], row['r8_text_content']) if row['text_change_type'] == '变更' and row['similar_score'] < 1 else '', axis=1)

------------------------------------------------------------
差异1：条款引用差异
R7：本条(a)款第(2)、(3)项
R8：本条(a)款第(2)项、第(3)项

无其他差异。
------------------------------------------------------------
差异1：主体对象差异
R7：多发飞机
R8：多发涡轮驱动飞机

差异2：主体对象差异
R7：多发飞机
R8：多发涡轮驱动飞机

差异3：主体对象差异
R7：多发飞机
R8：多发涡轮驱动飞机
------------------------------------------------------------
无内容差异
------------------------------------------------------------
无内容差异
------------------------------------------------------------
无内容差异
------------------------------------------------------------
无内容差异
------------------------------------------------------------
无内容差异
------------------------------------------------------------
差异1：主体对象差异
R7：飞机适航性
R8：飞机的适航性

无其他差异。
------------------------------------------------------------
无内容差异
------------------------------------------------------------
无内容差异
------------------------------------------------------------
差异1：主体对象差异
R7：无
R8：无

其他差异：上面提到的内容之外的差异（尽量简短描述，请不要重复上面提到的差异点）
R7：应符合
R8：应当符合
------------------

In [ ]:
src_df.to_excel('r7_r8_marked_diff_result.xlsx', index=False)

In [11]:
len(src_df)

1261